In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import model_summary as ms
import cv2
import keras
#from model import load_model
import numpy as np
import PIL
from sklearn.metrics import mean_squared_error
from keras.models import load_model
import keras
import pandas as pd
from google.colab.patches import cv2_imshow
import torch
import os
import shutil, sys
import datetime
import random

no_of_sample = 1

if no_of_sample == 1:
  threshold=0.138
  vc = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera1_left_near_optical_GH010024.mp4')
  vcThermal = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera1_left_near_thermal_ZK3X1s3g8B.mp4')

if no_of_sample == 2:
  threshold=0.12
  vc = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera2_right_near_optical_GH010141.mp4')
  vcThermal = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera2_right_near_thermal_wdPJujIkra.mp4')

if no_of_sample == 3:
  threshold=0.111
  vc = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera3_top_optical_GH010017.mp4')
  vcThermal = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera3_top_thermal_Y1NUdOUtCA.mp4')

if no_of_sample == 4:
  threshold=0.137
  vc = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera4_left_far_optical_GH010211.mp4')
  vcThermal = cv2.VideoCapture('/content/drive/MyDrive/METIS/videos/camera4_left_far_thermal_XBaOGrTo8r.mp4')

rval = True
previousFrameStatus = True

print('Loading autoencoder')
model = keras.models.load_model('/content/drive/MyDrive/METIS/models/abnormal_model_for_rgb_side_camera.h5')
modelThermal = keras.models.load_model('/content/drive/MyDrive/METIS/models/abnormal_model_thermal_side_camera.h5')

Loading autoencoder


# **Redis Implementation**

In [ ]:
!pip install redis
import redis
import time

r = redis.Redis(
    host = '147.102.19.87',
    port = 6379,
    password = 'metis')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 5.2 MB/s 


RGB input stream

In [ ]:
'''
score = 0 #input reconstruction error for RGB autoencoder
r.zadd('vision', {time.time(): score})
'''

"\nscore = 0 #input reconstruction error for RGB autoencoder\nr.zadd('vision', {time.time(): score})\n"

Thermal input stream

In [ ]:
'''
score = 0 #input reconstruction error for thermal autoencoder
r.zadd('thermal', {time.time(): score})
'''

"\nscore = 0 #input reconstruction error for thermal autoencoder\nr.zadd('thermal', {time.time(): score})\n"

Radar input stream

In [ ]:
'''
score = r.zrange('radar', time.time()-0.05, time.time(), desc=True, withscores=True)
threshold = 0

for i in score:
    if i[1]>max:
    max = i

if max > threshold:
  #call object detection method
'''

"\nscore = r.zrange('radar', time.time()-0.05, time.time(), desc=True, withscores=True)\nthreshold = 0\n\nfor i in score:\n    if i[1]>max:\n    max = i\n\nif max > threshold:\n  #call object detection method\n"

Object Detection Score

In [ ]:
'''
score = 0 #input reconstruction error for thermal autoencoder
r.zadd('object', {time.time(): score})
'''

"\nscore = 0 #input reconstruction error for thermal autoencoder\nr.zadd('object', {time.time(): score})\n"

Create Frames from Video

In [ ]:
frames = []
while(True): 
    ret,frame = vc.read() 
    if ret: 
        frames.append(frame)
    else: 
        break

In [ ]:
framesThermal = []
while(True): 
    ret,frame = vcThermal.read() 
    if ret: 
        framesThermal.append(frame)
    else: 
        break

In [ ]:
print(len(frames))
print(len(framesThermal))

600
600


Load YOLO

In [ ]:
print('Loading YOLO')
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/METIS/models/YOLO_optical_best.pt')  # default

!git clone https://github.com/ultralytics/yolov5

Loading YOLO


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-7-7 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


Cloning into 'yolov5'...
remote: Enumerating objects: 12412, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 12412 (delta 11), reused 17 (delta 4), pack-reused 12380
Receiving objects: 100% (12412/12412), 12.21 MiB | 29.90 MiB/s, done.
Resolving deltas: 100% (8554/8554), done.


Load Thermal YOLO

In [ ]:
print('Loading Thermal YOLO')
yolo_model_thermal = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/METIS/models/YOLO_thermal_best.pt')  # default

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


Loading Thermal YOLO


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-7-7 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


Make Predictions Autoencoder

In [ ]:
video = []
memory = 50 # memory equals 5sed times the framerate
alert_frame = 0
alert_in_the_last_5_sec = False
initiate_yolo = False
last_activated_yolo = -500
alert = False

for x in range(0,(len(frames)//10)*10,10): #while true for live video
	optical_alert = 0
	thermal_alert = 0
	yolo_alert = 0
	imagedump=[]
	framedump = []
	imagedumpThermal=[]
	framedumpThermal = []
	timestamp = time.time()

	for i in range(10):
		frame = frames[x+i]
		framedump.append(frame)
		#video.append(frame)
		frame = np.resize(frame, (240,240,3))

		#Convert the Image to Grayscale
		gray = 0.2989 * frame[:, :, 0] + 0.5870 * frame[:, :, 1] + 0.1140 * frame[:, :, 2]
		gray = (gray-gray.mean())/gray.std()
		gray = np.clip(gray,0,1)
		imagedump.append(gray)

	for i in range(10):
			frameThermal = framesThermal[x+i]
			framedumpThermal.append(frameThermal)
			#video.append(frame)
			frameThermal = np.resize(frame, (240,240,3))

			#Convert the Image to Grayscale
			gray = 0.2989 * frame[:, :, 0] + 0.5870 * frame[:, :, 1] + 0.1140 * frame[:, :, 2]
			gray = (gray-gray.mean())/gray.std()
			gray = np.clip(gray,0,1)
			imagedumpThermal.append(gray)

	imagedump = np.array(imagedump)
	imagedumpThermal = np.array(imagedumpThermal)

	imagedump.resize(240, 240, 10)
	imagedump = np.expand_dims(imagedump, axis=0)
	imagedump = np.expand_dims(imagedump, axis=4)

	imagedumpThermal.resize(240, 240, 10)
	imagedumpThermal = np.expand_dims(imagedumpThermal, axis=0)
	imagedumpThermal = np.expand_dims(imagedumpThermal, axis=4)
	#print('Processing data... ')
	#print('Autoencoder run: '+ str(int(x/10)))
	output = model.predict(imagedump)
	outputThermal = modelThermal.predict(imagedumpThermal)


	lossOptical = mean_squared_error(imagedump[0,:,:,0,0], output[0,:,:,0,0])
	r.zadd('optical', {timestamp: lossOptical})
	r.zadd('optical_alert', {timestamp: optical_alert})
 
	lossThermal = mean_squared_error(imagedumpThermal[0,:,:,0,0], outputThermal[0,:,:,0,0])
	r.zadd('thermal', {timestamp: lossThermal})
	r.zadd('thermal_alert', {timestamp: thermal_alert})


	if (alert == False):
		if lossOptical > threshold and previousFrameStatus:
			print('Anomalies Detected - Begin: '+str(x))
			previousFrameStatus = not previousFrameStatus
			video_first_frame = x - memory

			optical_alert = 1
			thermal_alert = 1
			#if len(video) > memory:
			#	video = video[-memory:] # keep last 1 second of video frames
		elif lossOptical > threshold and not previousFrameStatus:
			#print('Anomalies Detected - Continue')

			optical_alert = 1
			thermal_alert = 1
			#pass
		elif lossOptical < threshold and not previousFrameStatus:
			print('Return to normal state - Post end JSON and frames: '+str(x))
			previousFrameStatus = not previousFrameStatus
			#for j in range (memory):
			#	aftermath = frames[x]
			#	video.append(aftermath)
			video_last_first_frame = x
			if (video_first_frame<0):
				video_first_frame=0
			video = frames[video_first_frame:video_last_first_frame]
			print("Video length: "+str(len(video)))
			optical_alert = 0
			thermal_alert = 0
			#run YOLO on video
			initiate_yolo = True
	
	if (initiate_yolo == True):
		initiate_yolo = False
		print('Running YOLO')
		flag = False
		for yolo_frame in video:
			results = yolo_model(yolo_frame)
			df = results.pandas().xyxy[0]
			if (df.shape[0] != 0):
				flag = True
				break
		if flag == True:
			print("YOLO ACTIVATED AN ALERT! MOB on frame " + str(x))
			alert_frame = x
			alert = True
			video = []
			yolo_alert = 1
		else:
			print("YOLO WILL NOT ACTIVATE AN ALERT")

	if ((alert == True) and ((alert_frame+250 <= x) or (x == (len(frames)//10)*10)-10)):
		if x>500:
			first_yolo_frame = x-250
		else:
			first_yolo_frame = 0
		last_yolo_frame = x
		alert = False
		run_yolo_on_frames = frames[first_yolo_frame:last_yolo_frame]
		run_yolo_on_frames_thermal = framesThermal[first_yolo_frame:last_yolo_frame]




	r.zadd('yolo_alert', {timestamp: yolo_alert})

Anomalies Detected - Begin: 0
Return to normal state - Post end JSON and frames: 20
Video length: 20
Running YOLO
YOLO WILL NOT ACTIVATE AN ALERT
Anomalies Detected - Begin: 150
Return to normal state - Post end JSON and frames: 170
Video length: 70
Running YOLO
YOLO ACTIVATED AN ALERT! MOB on frame 170
Anomalies Detected - Begin: 180
Return to normal state - Post end JSON and frames: 370
Video length: 240
Running YOLO
YOLO ACTIVATED AN ALERT! MOB on frame 370
Anomalies Detected - Begin: 400
Return to normal state - Post end JSON and frames: 410
Video length: 60
Running YOLO
YOLO WILL NOT ACTIVATE AN ALERT
Anomalies Detected - Begin: 460
Return to normal state - Post end JSON and frames: 470
Video length: 60
Running YOLO
YOLO WILL NOT ACTIVATE AN ALERT
Anomalies Detected - Begin: 480
Return to normal state - Post end JSON and frames: 510
Video length: 80
Running YOLO
YOLO WILL NOT ACTIVATE AN ALERT


In [ ]:
print(r.zrange('thermal', 0, -1, desc=True, withscores=True))
print(r.zrange('optical_alert', 0, -1, desc=True, withscores=True))
print(r.zrange('thermal_alert', 0, -1, desc=True, withscores=True))
print(r.zrange('yolo_alert', 0, -1, desc=True, withscores=True))

[(b'1657016220.8302388', 0.380972349774059), (b'1657016231.525337', 0.37989851821862775), (b'1657103603.6944203', 0.37757342164983826), (b'1657103594.8534446', 0.35872011018843764), (b'1657103593.257203', 0.35810352072261775), (b'1657103599.571505', 0.3560729020358306), (b'1656977210.1784408', 0.3488724004780122), (b'1657103587.4159276', 0.3403992254176131), (b'1657103572.8303692', 0.33912801296190154), (b'1656977208.2972074', 0.33683792762578274), (b'1657103544.616569', 0.33488370339431994), (b'1657103582.7247458', 0.3341855841674742), (b'1657103581.3597012', 0.33026386195357765), (b'1657015821.694126', 0.32572346974769867), (b'1657015058.248435', 0.3257177486039608), (b'1657016242.1002276', 0.32424873523607745), (b'1657016238.1577773', 0.32250730607714334), (b'1657016213.3729398', 0.32061358616401403), (b'1657103571.4407291', 0.32045142470639565), (b'1656977175.3802876', 0.31973274903748267), (b'1657103551.1756551', 0.3184536757538518), (b'1657015823.0546045', 0.3184496174327965), (b

# Runs YOLO on frames and create clean video

In [ ]:
os.mkdir("/content/yolo_frames")
counter=1
for img in run_yolo_on_frames:
  cv2.imwrite("/content/yolo_frames/yolo_frame_"+str(counter)+".png", img)
  counter=counter+1
os.system("ffmpeg -r 50 -i /content/yolo_frames/yolo_frame_%01d.png -vcodec mpeg4 -y /content/sample_data/movie.mp4")
shutil.rmtree("/content/yolo_frames")

In [ ]:
os.mkdir("/content/yolo_frames")
counter=1
for img in run_yolo_on_frames_thermal:
  cv2.imwrite("/content/yolo_frames/yolo_frame_"+str(counter)+".png", img)
  counter=counter+1
os.system("ffmpeg -r 50 -i /content/yolo_frames/yolo_frame_%01d.png -vcodec mpeg4 -y /content/sample_data/movie_thermal.mp4")
shutil.rmtree("/content/yolo_frames")

Create Video from Frames

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/METIS/models/YOLO_optical_best.pt --img 640 --conf 0.25 --source /content/sample_data/movie.mp4

detect: weights=['/content/drive/MyDrive/METIS/models/YOLO_optical_best.pt'], source=/content/sample_data/movie.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-280-g27d831b Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
video 1/1 (1/371) /content/sample_data/movie.mp4: 640x384 Done. (0.021s)
video 1/1 (2/371) /content/sample_data/movie.mp4: 640x384 Done. (0.016s)
video 1/1 (3/371) /content/sample_data/movie.mp4: 640x384 Done. (0.010s)
video 1/1 (4/371) /content/sample_data/movie.mp4: 640x384 Done. (0.011s)
video 1/1 (5/371) /content/sam

In [ ]:
shutil.move("/content/yolov5/runs/detect/exp/movie.mp4", "/content/sample_data/movie.mp4")
shutil.rmtree("/content/yolov5/runs/detect/exp")

ct =str(int(time.time()))
os.rename("/content/sample_data/movie.mp4", "/content/sample_data/output_optical"+str(ct)+".mp4")
shutil.move("/content/sample_data/output_optical"+str(ct)+".mp4", "/content/drive/MyDrive/METIS/outputs/output_optical"+str(ct)+".mp4")
opticalVideoTimestamp = str(ct)

In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/METIS/models/YOLO_thermal_best.pt --img 320 --conf 0.25 --source /content/sample_data/movie_thermal.mp4

detect: weights=['/content/drive/MyDrive/METIS/models/YOLO_thermal_best.pt'], source=/content/sample_data/movie_thermal.mp4, data=yolov5/data/coco128.yaml, imgsz=[320, 320], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-280-g27d831b Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
video 1/1 (1/371) /content/sample_data/movie_thermal.mp4: 320x256 Done. (0.014s)
video 1/1 (2/371) /content/sample_data/movie_thermal.mp4: 320x256 Done. (0.009s)
video 1/1 (3/371) /content/sample_data/movie_thermal.mp4: 320x256 Done. (0.009s)
video 1/1 (4/371) /content/sample_data/movie_thermal.mp4: 320x256 Done.

In [ ]:
shutil.move("/content/yolov5/runs/detect/exp/movie_thermal.mp4", "/content/sample_data/movie_thermal.mp4")
shutil.rmtree("/content/yolov5/runs/detect/exp")

ct =str(int(time.time()))
os.rename("/content/sample_data/movie_thermal.mp4", "/content/sample_data/output_thermal"+str(ct)+".mp4")
shutil.move("/content/sample_data/output_thermal"+str(ct)+".mp4", "/content/drive/MyDrive/METIS/outputs/output_thermal"+str(ct)+".mp4")
thermalVideoTimestamp = str(ct)

In [ ]:
#Wait 10 seconds for the upload to Drive to be completed.
time.sleep(10)

In [ ]:
filenameOptical = "/content/drive/MyDrive/METIS/outputs/output_optical"+opticalVideoTimestamp+".mp4"
filenameThermal = "/content/drive/MyDrive/METIS/outputs/output_thermal"+thermalVideoTimestamp+".mp4"



In [ ]:
!pip install pysftp
import pysftp

cnopts = pysftp.CnOpts()
cnopts.hostkeys = None 
with pysftp.Connection('147.102.19.87', username='ntua', password='talampekos87@', port=22, cnopts=cnopts) as sftp:
  with sftp.cd('/home/ntua/Public/MetisVideoFiles/Optical'):
    sftp.put(filenameOptical)
    print('RGB File Uploaded')
  with sftp.cd('/home/ntua/Public/MetisVideoFiles/Thermal'):
    sftp.put(filenameOptical)
    print('Thermal File Uploaded')

opticalFileLocation = '/home/ntua/Public/MetisVideoFiles/Optical' + filenameOptical
thermalFileLocation = '/home/ntua/Public/MetisVideoFiles/Thermal' + filenameThermal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/pysftp/__init__.py:61: UserWarning: Failed to load HostKeys from /root/.ssh/known_hosts.  You will need to explicitly load HostKeys (cnopts.hostkeys.load(filename)) or disableHostKey checking (cnopts.hostkeys = None).
  warnings.warn(wmsg, UserWarning)
Connected (version 2.0, client OpenSSH_8.2p1)
Authentication (password) successful!
[chan 0] Opened sftp connection (server version 3)


RGB File Uploaded
Thermal File Uploaded


[chan 0] sftp session closed.


In [ ]:
# filename = "/content/drive/MyDrive/METIS/outputs/output_thermal"+thermalVideoTimestamp+".mp4"
# # Retrieving the file ID for a file in `"/content/drive/My Drive/"`:
# id = !xattr -p 'user.drive.id' {filename}
# print(id)
# print("https://drive.google.com/file/d/"+ id[0] + "//view?usp=sharing")
# fileLocationThermal  = "https://drive.google.com/file/d/"+ id[0] + "//view?usp=sharing"

['1-6SUhPeT0p3WJbzKSrgc-xLG7QYR6mbV']
https://drive.google.com/file/d/1-6SUhPeT0p3WJbzKSrgc-xLG7QYR6mbV//view?usp=sharing


In [ ]:
print(fileLocationOptical)
print(fileLocationThermal)

r.zadd('thermal_video', {fileLocationThermal: timestamp })
r.zadd('optical_video', {fileLocationOptical: timestamp })

print(r.zrange('thermal_video', 0, -1, desc=True, withscores=True))
print(r.zrange('optical_video', 0, -1, desc=True, withscores=True))

https://drive.google.com/file/d/1o2fDHgQwb5ZtZVHHgqtVORnrUq64uy8s//view?usp=sharing
https://drive.google.com/file/d/1-6SUhPeT0p3WJbzKSrgc-xLG7QYR6mbV//view?usp=sharing
[(b'https://drive.google.com/file/d/1-6SUhPeT0p3WJbzKSrgc-xLG7QYR6mbV//view?usp=sharing', 1657214706.4727147), (b'https://drive.google.com/file/d/1-3xdxoZpqorVAT8Lwi79e5WRudTLe8w7//view?usp=sharing', 1657200611.8300571)]
[(b'https://drive.google.com/file/d/1o2fDHgQwb5ZtZVHHgqtVORnrUq64uy8s//view?usp=sharing', 1657214706.4727147), (b'https://drive.google.com/file/d/1bj4uFHfB4futkSg2Odw3lKJW1yMjJVpd//view?usp=sharing', 1657200611.8300571)]


output_optical1657011198.mp4  output_thermal1657011210.mp4
output_optical1657013723.mp4  output_thermal1657013747.mp4
output_optical1657015886.mp4  output_thermal1657015897.mp4
output_optical1657103709.mp4  output_thermal1657103732.mp4
output_optical1657200690.mp4  output_thermal1657200707.mp4
